# Create files with index and generation data at state level

In [177]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
from os.path import join
import glob
import numpy as np
from joblib import Parallel, delayed
import sys
cwd = os.getcwd()
data_path = join(cwd, '..', 'Data storage')
idx = pd.IndexSlice

In [4]:
# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

In [7]:
# add the 'src' directory as one where we can import modules
src_dir = join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

In [210]:
%aimport Analysis.index
from Analysis.index import facility_emission_gen, group_facility_data
%aimport Analysis.index
from Analysis.index import facility_co2, adjust_epa_emissions, group_fuel_cats
%aimport Analysis.index
from Analysis.index import extra_emissions_gen, add_datetime, add_quarter
%aimport util.utils
from util.utils import rename_cols, add_facility_location

In [10]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE",
          "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS",
          "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS",
          "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY",
          "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

## Load data

Emission factors

In [43]:
path = join(data_path, 'Final emission factors.csv')
ef = pd.read_csv(path, index_col=0)

EIA facility data and EPA monthly emissions

In [13]:
facility_path = join(data_path, 'Facility gen fuels and CO2 2017-08-31.zip')
facility_df = pd.read_csv(facility_path)
facility_df['state'] = facility_df.geography.str[-2:]
rename_cols(facility_df)

epa_path = join(data_path, 'Derived data',
                'Monthly EPA emissions 2017-08-31.csv')
epa_df = pd.read_csv(epa_path)
rename_cols(epa_df)
facility_locations = pd.read_csv(join(data_path, 'Facility labels',
                                      'Facility locations.csv'))
# Add state labels to the EPA facilities
epa_df = add_facility_location(epa_df, facility_locations, labels=['state'])

In [24]:
facility_df.head()

,f,fuel,month,plant id,total fuel (mmbtu),year,generation (mwh),elec fuel (mmbtu),geography,last_updated,lat,lon,prime mover,datetime,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg),state
0,M,SUB,6,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-06-01,2,0.0,0.0,0.0,0.0,WI
1,M,SUB,5,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-05-01,2,0.0,0.0,0.0,0.0,WI
2,M,SUB,4,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-04-01,2,0.0,0.0,0.0,0.0,WI
3,M,SUB,3,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-03-01,1,0.0,0.0,0.0,0.0,WI
4,M,SUB,2,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-02-01,1,0.0,0.0,0.0,0.0,WI


In [18]:
epa_df.head()

,co2_mass (kg),gload (mw),heat_input (mmbtu),month,op_time,plant id,sload (1000lb/hr),year,state
0,9.625411e+08,1167596.0,1.113000e+07,1,4348.25,3,NaN,2001,AL
1,8.208495e+08,880517.0,8.951275e+06,2,3391.00,3,NaN,2001,AL
2,6.402928e+08,735523.0,7.175850e+06,3,3846.50,3,NaN,2001,AL
3,7.417879e+08,898023.0,8.519262e+06,4,4292.75,3,NaN,2001,AL
4,9.767243e+08,1230441.0,1.138663e+07,5,5029.75,3,NaN,2001,AL


JSON files with fuel categories

In [143]:
fuel_cat_folder = join(data_path, 'Fuel categories')
state_cats_path = join(fuel_cat_folder, 'State_facility.json')

with open(state_cats_path, 'r') as f:
    state_fuel_cat = json.load(f)
    
custom_cats_path = join(fuel_cat_folder, 'Custom_results.json')
with open(custom_cats_path, 'r') as f:
    custom_fuel_cat = json.load(f)

EIA total monthly gen and fuel consumption

In [266]:
eia_totals = pd.read_csv(join(data_path, 'EIA state-level gen fuel CO2.csv'),
                         parse_dates=['datetime'])
rename_cols(eia_totals)
eia_totals['state'] = eia_totals.series_id.str.split('-').str[-2]
eia_totals['type'] = (eia_totals.series_id.str
                      .split('-').str[0].str
                      .split('.').str[-1]
                      )
eia_totals['year'] = eia_totals.datetime.dt.year
eia_totals['month'] = eia_totals.datetime.dt.month
eia_totals = eia_totals.loc[~eia_totals.type.isin(['SPV', 'AOR', 'TSN'])]

In [155]:
eia_totals.head()

,end,f,last_updated,sector,series_id,start,units,generation (mwh),total fuel (mmbtu),elec fuel (mmbtu),all fuel co2 (kg),elec fuel co2 (kg),datetime,quarter,state,type,year,month
9826,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.COW-AK-99.M,200101.0,megawatthours,46903.0,1120000.0,872000.0,1.066800e+08,8.305800e+07,2001-01-01,1,AK,COW,2001,1
9827,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.COW-AL-99.M,200101.0,megawatthours,6557913.0,67999000.0,66582000.0,6.476905e+09,6.341936e+09,2001-01-01,1,AL,COW,2001,1
9828,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.COW-AR-99.M,200101.0,megawatthours,2149808.0,23099000.0,22700000.0,2.200180e+09,2.162175e+09,2001-01-01,1,AR,COW,2001,1
9829,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.COW-AZ-99.M,200101.0,megawatthours,3418454.0,35873000.0,35483000.0,3.416903e+09,3.379756e+09,2001-01-01,1,AZ,COW,2001,1
9830,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.COW-CA-99.M,200101.0,megawatthours,199857.0,3652000.0,2008000.0,3.478530e+08,1.912620e+08,2001-01-01,1,CA,COW,2001,1


## Calculate state-level monthly CO₂ intensity and generation by fuel category

In [271]:
index_list = []
gen_list = []
for state in states:
    eia_fac_state = facility_df.loc[facility_df.state == state].copy()
    eia_totals_state = eia_totals.loc[eia_totals.state == state].copy()
    epa_state = epa_df.loc[epa_df.state == state].copy()
    

    co2, gen_fuels_state = facility_emission_gen(eia_facility=eia_fac_state,
                                                 epa=epa_state,
                                                 state_fuel_cat=state_fuel_cat,
                                                 custom_fuel_cat=custom_fuel_cat,
                                                 export_state_cats=True,
                                                 print_status=False)
    
    extra_co2, extra_gen = extra_emissions_gen(gen_fuels_state,
                                               eia_totals_state, ef)
    
    # Combine facility and extra co2, name the series
    co2_monthly = co2.groupby(['year', 'month']).sum()
    total_co2 = (co2_monthly.loc[:, 'final co2 (kg)']
                 + extra_co2.loc[:, 'elec fuel co2 (kg)']
                            .groupby(['year', 'month']).sum())
    total_co2.name = 'final co2 (kg)'
    
    # Total gen, and the co2 intensity
    total_gen = (eia_totals_state
                 .groupby(['year', 'month'])['generation (mwh)'].sum())
    
    state_index = pd.concat([total_co2, total_gen], axis=1)
    state_index['index (g/kwh)'] = (state_index['final co2 (kg)']
                                    / state_index['generation (mwh)'])
    state_index['state'] = state
    state_index.set_index('state', append=True, inplace=True)
    
    # Generation by fuel category
    gen_category = group_fuel_cats(eia_totals_state, custom_fuel_cat, 
                                   fuel_col='type', new_col='fuel category')
    
    keep_cols = ['fuel category', 'generation (mwh)', 'total fuel (mmbtu)',
                 'elec fuel (mmbtu)', 'all fuel co2 (kg)',
                 'elec fuel co2 (kg)', 'year', 'month']
    gen_category = gen_category[keep_cols]
    gen_category['state'] = state
    gen_category.set_index(['year', 'month', 'state'], inplace=True)
    
    # Add each df to the list
    index_list.append(state_index)
    gen_list.append(gen_category)


# Combine lists of dataframes
state_index_all = pd.concat(index_list)
add_quarter(state_index_all)

gen_category_all = pd.concat(gen_list)
add_quarter(gen_category_all)

# output state results to file
index_fn = 'Monthly index states.csv'
gen_fn = 'Monthly generation states.csv'
    
state_index_all.to_csv(join(data_path, 'final state data', index_fn))
gen_category_all.to_csv(join(data_path, 'final state data', gen_fn))